<a href="https://colab.research.google.com/github/Kerriea-star/KerasNLP/blob/main/Getting_Started_with_KerasNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

KerasNLP is a natural language processing library that supports users through their entire development cycle. Our workflows are built from modular components that have state-of-the-art preset weights and architectures when used out-of-the-box and are easily customizable when more control is needed.

This library is an extension of the core Keras API; all high-level modules are Layers or Models.

KerasNLP uses the **Keras Core** library to work with any of TensorFlow, Pytorch and Jax. In the guide below, we will use the `jax` backend for training our models, and `tf.data` for efficiently running our input preprocessing. But feel free to mix things up! This guide runs in TensorFlow or PyTorch backends with zero changes, simply update the `KERAS_BACKEND` below.


Search Keras documentation...

» Developer guides / KerasNLP / Getting Started with KerasNLP
Getting Started with KerasNLP
Author: Jonathan Bischof
Date created: 2022/12/15
Last modified: 2023/07/01
Description: An introduction to the KerasNLP API.

 View in Colab • GitHub source

Introduction
KerasNLP is a natural language processing library that supports users through their entire development cycle. Our workflows are built from modular components that have state-of-the-art preset weights and architectures when used out-of-the-box and are easily customizable when more control is needed.

This library is an extension of the core Keras API; all high-level modules are Layers or Models. If you are familiar with Keras, congratulations! You already understand most of KerasNLP.

KerasNLP uses the Keras Core library to work with any of TensorFlow, Pytorch and Jax. In the guide below, we will use the jax backend for training our models, and tf.data for efficiently running our input preprocessing. But feel free to mix things up! This guide runs in TensorFlow or PyTorch backends with zero changes, simply update the KERAS_BACKEND below.

This guide demonstrates our modular approach using a sentiment analysis example at six levels of complexity:

*   Inference with a pretrained classifier

*   Fine tuning a pretrained backbone
*   Fine tuning with user-controlled preprocessing
*   Fine tuning a custom model
*   Pretraining a backbone model
*   LBuild and train your own transformer from scratch

Throughout our guide, we use Professor Keras, the official Keras mascot, as a visual
reference for the complexity of the material:

<img src="https://storage.googleapis.com/keras-nlp/getting_started_guide/prof_keras_evolution.png" alt="drawing" height="250"/>

In [1]:
!pip install -q --upgrade keras-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.1/590.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 25.5 MB/s eta 0:00:00


In [2]:
import os

os.environ["KERAS_BACKEND"] = "jax" # or "tensorflow" or "torch"

import keras_nlp
import keras_core as keras

Using JAX backend.


## API quickstart

Our highest level API is `keras_nlp.models`. These symbols cover the complete user
journey of converting strings to tokens, tokens to dense features, and dense features to
task-specific output. For each `XX` architecture (e.g., `Bert`), we offer the following
modules:

* **Tokenizer**: `keras_nlp.models.XXTokenizer`
  * **What it does**: Converts strings to sequences of token ids.
  * **Why it's important**: The raw bytes of a string are too high dimensional to be useful
    features so we first map them to a small number of tokens, for example `"The quick brown
    fox"` to `["the", "qu", "##ick", "br", "##own", "fox"]`.
  * **Inherits from**: `keras.layers.Layer`.
* **Preprocessor**: `keras_nlp.models.XXPreprocessor`
  * **What it does**: Converts strings to a dictionary of preprocessed tensors consumed by
    the backbone, starting with tokenization.
  * **Why it's important**: Each model uses special tokens and extra tensors to understand
    the input such as delimiting input segments and identifying padding tokens. Padding each
    sequence to the same length improves computational efficiency.
  * **Has a**: `XXTokenizer`.
  * **Inherits from**: `keras.layers.Layer`.
* **Backbone**: `keras_nlp.models.XXBackbone`
  * **What it does**: Converts preprocessed tensors to dense features. *Does not handle
    strings; call the preprocessor first.*
  * **Why it's important**: The backbone distills the input tokens into dense features that
    can be used in downstream tasks. It is generally pretrained on a language modeling task
    using massive amounts of unlabeled data. Transferring this information to a new task is a
    major breakthrough in modern NLP.
  * **Inherits from**: `keras.Model`.
* **Task**: e.g., `keras_nlp.models.XXClassifier`
  * **What it does**: Converts strings to task-specific output (e.g., classification
    probabilities).
  * **Why it's important**: Task models combine string preprocessing and the backbone model
    with task-specific `Layers` to solve a problem such as sentence classification, token
    classification, or text generation. The additional `Layers` must be fine-tuned on labeled
    data.
  * **Has a**: `XXBackbone` and `XXPreprocessor`.
  * **Inherits from**: `keras.Model`.

Here is the modular hierarchy for `BertClassifier` (all relationships are compositional):

<img src="https://storage.googleapis.com/keras-nlp/getting_started_guide/class_diagram.png" alt="drawing" height="300"/>

All modules can be used independently and have a `from_preset()` method in addition to
the standard constructor that instantiates the class with **preset** architecture and
weights (see examples below).

## Data
We will use a running example of sentiment analysis of IMDB movie reviews. In this task, we use the text to predict whether the review was positive (`label = 1`) or negative (`label = 0`).

We load the data using `keras.utils.text_dataset_from_directory`, which utilizes the powerful `tf.data.Dataset` format for examples.

In [3]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!# Remove unsupervised examples
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  46.6M      0  0:00:01  0:00:01 --:--:-- 46.6M


In [4]:
BATCH_SIZE = 16
imdb_train = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size = BATCH_SIZE,
)
imdb_test = keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size = BATCH_SIZE,
)

# Inspec first review
# Format is (review text tensor, label tensor)

print(imdb_train.unbatch().take(1).get_single_element())

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
(<tf.Tensor: shape=(), dtype=string, numpy=b"I very much looked forward to this movie. Its a good family movie; however, if Michael Landon Jr.'s editing team did a better job of editing, the movie would be much better. Too many scenes out of context. I do hope there is another movie from the series, they're all very good. But, if another one is made, I beg them to take better care at editing. This story was all over the place and didn't seem to have a center. Which is unfortunate because the other movies of the series were great. I enjoy the story of Willie and Missy; they're both great role models. Plus, the romantic side of the viewers always enjoy a good love story.">, <tf.Tensor: shape=(), dtype=int32, numpy=0>)


## Inference with a pretrained classifier

<img src="https://storage.googleapis.com/keras-nlp/getting_started_guide/prof_keras_beginner.png" alt="drawing" height="250"/>

The highest level module in KerasNLP is a **task**. A **task** is a `keras.Model`
consisting of a (generally pretrained) **backbone** model and task-specific layers.
Here's an example using `keras_nlp.models.BertClassifier`.

**Note**: Outputs are the logits per class (e.g., `[0, 0]` is 50% chance of positive). The output is
[negative, positive] for binary classification.

In [5]:
classifier = keras_nlp.models.BertClassifier.from_preset("bert_tiny_en_uncased_sst2")
# Note: batched inputs expected so must wrap string in iterable
classifier.predict(["I love modular workflows in keras-nlp!"])

231508/231508 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step       


17596448/17596448 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


array([[-1.5387032,  1.5418268]], dtype=float32)

All tasks have a `from_preset` method that constructs a `keras.Model` instance with preset preprocessing, architecture and weights. This means that we can pass raw strings in any format accepted by a `keras.Model` and get output specific to our task.

This particular preset is a `"bert_tiny_uncased_en"` backbone fine-tuned on `sst2`, another movie review sentiment analysis (this time from Rotten Tomatoes). We use the `tiny` architecture for demo purposes, but larger models are recommended for SoTA performance.

Let's evaluate our classifier on the IMDB dataset. You will note we don't need to call `keras.Model.compile` here. All task models like `BertClassifier` ship with compilation defaults, meaning we can just call `keras.Model.evaluate` directly. You can always call compile as normal to override these defaults (e.g. to add new metrics).

The output below is [loss, accuracy],

In [6]:
classifier.evaluate(imdb_test)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 683s 435ms/step - loss: 0.4666 - sparse_categorical_accuracy: 0.7821


[0.4629073441028595, 0.783519983291626]

## Fine tuning a pretrained BERT backbone

<img src="https://storage.googleapis.com/keras-nlp/getting_started_guide/prof_keras_intermediate.png" alt="drawing" height="250"/>

When labeled text specific to our task is available, fine-tuning a custom classifier can
improve performance. If we want to predict IMDB review sentiment, using IMDB data should
perform better than Rotten Tomatoes data! And for many tasks, no relevant pretrained model
will be available (e.g., categorizing customer reviews).

The workflow for fine-tuning is almost identical to above, except that we request a
**preset** for the **backbone**-only model rather than the entire classifier. When passed
a **backbone** **preset**, a **task** `Model` will randomly initialize all task-specific
layers in preparation for training. For all the **backbone** presets available for
`BertClassifier`.

To train your classifier, use `keras.Model.fit` as with any other
`keras.Model`. As with our inference example, we can rely on the compilation
defaults for the **task** and skip `keras.Model.compile`. As preprocessing is
included, we again pass the raw data.

In [8]:
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes = 2,
)
classifier.fit(
    imdb_train,
    validation_data=imdb_test,
    epochs=1
)

231508/231508 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step       
17602216/17602216 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3099s 2s/step - loss: 0.5275 - sparse_categorical_accuracy: 0.7170 - val_loss: 0.3016 - val_sparse_categorical_accuracy: 0.8728


Here we see a significant lift in validation accuracy (0.78 -> 0.87) with a single epoch of training even though the IMDB dataset is much smaller than `sst2`.